In [ ]:
from utils import *
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
from tqdm import tqdm, trange
import json
import torch

In [ ]:
# Load the model

device = "cuda"

model_path = "/root/annotated_textual_inversion/model/sd-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe = pipe.to(device)
pipe.safety_checker = lambda images, clip_input: (images, False)

In [ ]:
# Load the embedding

token = "<Monet>"
number = 1
weight_dir="/root/annotated_textual_inversion/embeds/Monet-TI-2000" # Monet-MTI-2-1200; Monet-ATI-1200; Monet-TI-2000
name="learned_embeds"

# for MTI
# token = load_MTI_embeds(pipe, model_path, token, number, weight_dir, name)

# for TI
pipe.load_textual_inversion(model_path, token, weight_name=os.path.join(weight_dir, f"{name}.bin"))

In [ ]:
# set parameters

image_path = "/root/annotated_textual_inversion/data/style/palace.jpg"
init_image = Image.open(image_path).convert("RGB")
init_image = init_image.resize((512, 512))

prompt = f"{token} style, a palace of Forbidden City" # a road lined with pink flowers; waterfall; a palace of Forbidden City
output_dir = f"/root/annotated_textual_inversion/output/samples"
num_samples = 10
step = 50
guidance_scale = 8
generator = torch.Generator(device="cuda").manual_seed(0)

In [ ]:
os.makedirs(output_dir, exist_ok=True)
pipe.set_progress_bar_config(disable=False)

sample = pipe(prompt=prompt, image=init_image, num_inference_steps=step, guidance_scale=guidance_scale, generator=generator).images
sample[0].save(f"{output_dir}/sample.png")

In [ ]:
# generate samples

pipe.set_progress_bar_config(disable=True)
os.makedirs(output_dir, exist_ok=True)

print(f"generating samples ...")
for i in trange(num_samples):
    sample = pipe(prompt=prompt, image=init_image, num_inference_steps=step, guidance_scale=guidance_scale, generator=generator).images
    sample[0].save(f"{output_dir}/sample-{i}.png")

In [1]:
import torch
import json
from utils import *
from transformers import AutoProcessor, CLIPModel

d:\anaconda3\envs\ldm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda")

model = CLIPModel.from_pretrained("./model/clip-vit-large-patch14/").to(device)
processor = AutoProcessor.from_pretrained("./model/clip-vit-large-patch14/")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [12]:
# validate image pair

target_name = "waterfall-512.jpg"
sample_name = "TI.png" # ATI.png; MTI.png; TI.png
dir = "waterfall"

target = f"./data/style/{dir}/{target_name}"
sample = f"./data/style/{dir}/{sample_name}"

clip_validate_image_pair(model, processor, sample, target, device=device)

0.5346982479095459

In [21]:
# validate text pair

sample_name = "MTI.png" # ATI.png; MTI.png; TI.png
dir = "waterfall"

text = "a painting in the style of Monet"
sample = f"./data/style/{dir}/{sample_name}"

clip_validate_text_image_pair(model, processor, text, sample, device=device)

0.2521168887615204